## Installing required libraries

In [13]:
!pip install --quiet --upgrade langchain langchain-community gradio pypdf langchain-openai faiss-cpu langchain-google-genai pillow

## loading credentials from .env

In [14]:
# import os
# from dotenv import load_dotenv

# load_dotenv('azure_credentials.env')

# EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")
# EMBEDDING_ENDPOINT = os.getenv("EMBEDDING_ENDPOINT")
# EMBEDDING_API_VERSION = os.getenv("EMBEDDING_API_VERSION")
# EMBEDDING_API_KEY = os.getenv("EMBEDDING_API_KEY")
# AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
# API_VERSION = os.getenv("API_VERSION")
# AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")

In [38]:
from google.colab import userdata

EMBEDDING_MODEL_NAME = userdata.get("EMBEDDING_MODEL_NAME")
EMBEDDING_ENDPOINT = userdata.get("EMBEDDING_ENDPOINT")
EMBEDDING_API_VERSION = userdata.get("EMBEDDING_API_VERSION")
EMBEDDING_API_KEY = userdata.get("EMBEDDING_API_KEY")
AZURE_OPENAI_API_KEY = userdata.get("AZURE_OPENAI_API_KEY")
API_VERSION = userdata.get("api_version")
AZURE_ENDPOINT = userdata.get("AZURE_ENDPOINT")

In [39]:
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

## Importing required libraries

In [40]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import RunnableParallel

from langchain_core.prompts import ChatPromptTemplate

from langchain_community.document_loaders import PyPDFLoader

from langchain_google_genai import ChatGoogleGenerativeAI

from utils import *

## Defining LLMs

In [41]:
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )

llm_2 = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )

gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [19]:
embeddings = AzureOpenAIEmbeddings(
    model=EMBEDDING_MODEL_NAME,
    azure_endpoint= EMBEDDING_ENDPOINT,
    api_key= EMBEDDING_API_KEY,
    openai_api_version= EMBEDDING_API_VERSION
)

## Prompts

In [43]:
from utils import prompt_ as rag_prompt, simple_prompt

## Getting Vector DB's Retriever

In [44]:
retriever = await get_retriever(["book_name.pdf"], embeddings=embeddings)

## Chains

In [45]:
from utils import get_rag_chain


# Define the second chain with LLM 2
rag_chain_1 = get_rag_chain(retriever, llm, rag_prompt)
rag_chain_2 = get_rag_chain(retriever, llm_2, rag_prompt)
gemini_chain = get_rag_chain(retriever, gemini_llm, simple_prompt)

# Define the third chain with LLM, without RAG
chain3 = (
    simple_prompt
    | llm_2  # Second LLM instance
    | StrOutputParser()
)

chain4 = (
    simple_prompt
    | gemini_llm  # Second LLM instance
    | StrOutputParser()
)

# Run all chains in parallel with RunnableParallel
parallel_chain = RunnableParallel(
    {"GPT_4o_MINI": rag_chain_1, "GPT_4o": rag_chain_2,
     "GP 4o Raw": chain3, "Gemini Rag":gemini_chain,"Gemini":chain4 }
)


# Gradio

In [46]:
import gradio as gr

# List for storing prev asked questions

In [47]:
qa_history = []

In [48]:
# Function to call the chains and storing the answers in list.
def generate_response(message, _):
    global Questions_Answers
    responses = parallel_chain.invoke(message)
    qa_history.append({"question": message, "response": responses})

    return responses["GPT_4o_MINI"], responses["GPT_4o"], responses["GP 4o Raw"], responses["Gemini Rag"], responses["Gemini"]

In [49]:
# Function to filter previous responses based on a search term
def search_history(search_term):
    # If search_term is empty, return all questions
    if search_term == "":
        return [(qa["question"], qa["response"]) for i, qa in enumerate(qa_history)]

    # Filter questions that contain the search term
    filtered_history = [(qa["question"], str(qa["response"])) for qa in qa_history if search_term.lower() in qa["question"].lower()]
    return filtered_history

### Gradio Interface

In [50]:

# Gradio Interface for Chatbot Tab
with gr.Blocks(fill_width=True, fill_height=True) as chatbot_tab:
    gr.Markdown("### Chatbot")

    # Input box for user question
    question_input = gr.Textbox(label="Ask a question")


    gr.HTML("<hr>")
    gr.Markdown("GPT 4o Mini RAG response")
    response1 = gr.Markdown("")


    gr.HTML("<hr>")
    gr.Markdown("GPT 4o RAG response")
    response2 = gr.Markdown("")

    gr.HTML("<hr>")
    gr.Markdown("GPT 4o response")
    response3 = gr.Markdown("")

    gr.HTML("<hr>")
    gr.Markdown("Gemini RAG Response")
    response4 = gr.Markdown("")

    gr.HTML("<hr>")
    gr.Markdown("Gemini")
    response5 = gr.Markdown("")

    gr.HTML("<hr>")

    # Button to submit question
    submit_button = gr.Button("Submit")

    # Connect question input to chatbot function and display response
    submit_button.click(fn=generate_response,
                        inputs=question_input, outputs=[response1, response2, response3, response4, response5])


# Gradio Interface for Previous Responses Tab
with gr.Blocks() as history_tab:
    gr.Markdown("### Previous Responses")

    # Search bar to filter questions
    search_input = gr.Textbox(label="Search questions")

    # Display previous questions in rows
    questions_output = gr.Dataframe(headers=["Question", "Response"], interactive=False)

    # Update questions output based on search term
    search_input.change(fn=search_history, inputs=search_input, outputs=questions_output)


# Combine the tabs into a single interface
with gr.Blocks() as app:
    with gr.Tabs():
        with gr.TabItem("Chatbot"):
            chatbot_tab.render()

        with gr.TabItem("Previous Responses"):
            history_tab.render()

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:999: UserWarning: Expected 2 arguments for function <function generate_response at 0x7a5b8da64b80>, received 1.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1003: UserWarning: Expected at least 2 arguments for function <function generate_response at 0x7a5b8da64b80>, received 1.
  warnings.warn(


In [51]:
app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3d3bee6da9cfbff7f1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
